In [127]:
import numpy as np
import pandas as pd
import math
import sys

- Default value:

In [128]:
MAX_ELEMENT = 200
SIZE_ELEMENT = 23
MAX_DEPTH = 1500
EXPONENT = 1

sys.setrecursionlimit(10000)

- Read data:

In [129]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

X_train = train.iloc[:, :-1].to_numpy()
y_train = train.iloc[:, -1].to_numpy()

X_test = test.iloc[:, :-1].to_numpy()
y_test = test.iloc[:, -1].to_numpy()

- Fitness function:

In [130]:
def find_weight(X, y):
    return np.linalg.inv(X.T @ X) @ X.T @ y

def calc_y_predict(X, w):
    return np.sum(X * w.T, axis = 1)

def MAE(y_predict, y):
    return np.mean(np.abs(y_predict - y))

def fitness_function(ind):
    w = find_weight(X_train[:, ind] ** EXPONENT, y_train)
    y_predict = calc_y_predict(X_test[:, ind] ** EXPONENT, w)
    return MAE(y_predict, y_test) 

- Check if `child` already in `population`:

In [131]:
def IsInAll(arr, element):
    for x in arr:
        if (np.array_equal(x, element)):
            return True
        
    return False

- Generate sample:

In [132]:
def init():
    with open("genetic.txt", "r") as f:
        state = np.fromstring(f.readline()[1:-1], dtype = np.int16, sep = " ").astype(bool)

    sample = [state]

    while (len(sample) < MAX_ELEMENT):
        child = np.random.randint(2, size=SIZE_ELEMENT).astype(bool)

        if (not IsInAll(sample, child)):
            sample.append(child)

    sample.sort(key = fitness_function)

    return sample

- Genetic Algorithm:

In [133]:
def GeneticAlgorithm(sample, index = 0):
    if (index == MAX_DEPTH):
        return
    
    with open("genetic.txt", "w") as f:
        state = sample[0].astype(np.int16)
        f.write(np.array2string(state, separator = " "))
    
    if (index % 100 == 0):
        print("Generation: ", index)
        
    next_generation = []

    # 10% of the population are the elite
    for i in range(0, math.ceil(0.1 * MAX_ELEMENT)):
        next_generation.append(sample[i])

    # Create 90% of the population by crossover
    # 50% of the best population are the parents
    for i in range(0, math.ceil(0.9 * MAX_ELEMENT)):
        parent1 = sample[np.random.randint(0, 0.5 * MAX_ELEMENT)]
        parent2 = sample[np.random.randint(0, 0.5 * MAX_ELEMENT)]

        # Crossover
        child = np.zeros_like(parent1)

        for x in range(0, len(child)):
            rate = np.random.rand()

            if (rate < 0.45):
                child[x] = parent1[x]
            elif (rate < 0.9):
                child[x] = parent2[x]
            else:
                child[x] = np.random.randint(0, 2)

        # Add child to the next generation
        #if (not IsInAll(next_generation, child)):
        #    next_generation.append(child)

    # If the population is not enough
    while (len(next_generation) < MAX_ELEMENT):
        isInArr = False
        child = np.random.randint(2, size=SIZE_ELEMENT).astype(bool)

        if (not IsInAll(next_generation, child)):
            next_generation.append(child)
    
    # Sort the population by fitness function
    sample = sorted(sample[:MAX_ELEMENT], key = fitness_function)
    
    GeneticAlgorithm(next_generation, index + 1)

In [134]:
#sample = init()
#GeneticAlgorithm(sample)
#print("Best solution: ", sample[0])
#print("MAE: ", fitness_function(sample[0]))

In [135]:
model2 = ["Gender", "CollegeTier", "Degree", "CollegeCityTier","CivilEngg"]
model2 = ["10percentage", "12percentage", "collegeGPA", "English", "Logical", "Quant", "Domain", "ComputerProgramming", "ElectronicsAndSemicon", "ComputerScience", "MechanicalEngg", "ElectricalEngg", "TelecomEngg", "CivilEngg", "conscientiousness", "agreeableness", "extraversion", "nueroticism", "openess_to_experience"]
X = train.loc[:, model2].to_numpy()
w = find_weight(X ** EXPONENT, y_train)
y_predict = calc_y_predict(test.loc[:, model2].to_numpy() ** EXPONENT, w)
print(MAE(y_predict, y_test))
print(len(model2))

103903.30244282905
19
